# Rertival of kenya paper using R code (easypubmed and bash)
we were to obtain kenya papers from the ncbi database using an R code [easypubmed](https://github.com/dami82/easyPubMed) designed by Damiano Fantini. The code queries the database using the [entrez] (https://www.ncbi.nlm.nih.gov/books/NBK25497/) programming utilies. However the author was able to introduces more functions that can assist in easy manipulation of data. Hence, using the varies functions from the package we set out to accomplish the following objective 
1. To retrieve Kenya papers from ncbi ranging from the year 1980-2020 
2. TO retrieve the status of open access in kenya  
3. To determine the affliation of each paper (Determine the collaboration status among Kenya authors)

In [ ]:
# installing packages 
install.packages("easyPubMed")
install.packages("devtools")

In [1]:
#load library packages R
#setting the environment
getOption("unzip")
Sys.getenv("TAR") 
options(unzip = "/opt/conda/bin/unzip")
Sys.setenv(TAR = "/bin/tar")
devtools::install_github("dami82/easyPubMed")
devtools::install_github("dami82/businessPubMed")

[1] "/usr/bin/unzip"

[1] "/bin/gtar"

Skipping install of 'easyPubMed' from a github remote, the SHA1 (a2b7ffc8) has not changed since last install.
  Use `force = TRUE` to force installation
Skipping install of 'businessPubMed' from a github remote, the SHA1 (83798237) has not changed since last install.
  Use `force = TRUE` to force installation


In [2]:
#load the pakages in R
library(devtools)
library(easyPubMed)
library(rentrez)
library(kableExtra)
library(dplyr)

Loading required package: usethis

Attaching package: ‘dplyr’

The following object is masked from ‘package:kableExtra’:

    group_rows

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
##load the pakages in R
library("businessPubMed")
library("githubinstall")

Loading required package: XML


In [4]:
#Designing the query for ncbi
my.query <- paste('Kenya[Affiliation]')
my.query <- paste(my.query, 'AND ("1980"[EDAT]:"2019"[EDAT])')

In [5]:
# Show query string
cat(my.query)

Kenya[Affiliation] AND ("1980"[EDAT]:"2019"[EDAT])

In [6]:
# define the regex filter for countries other than kenya 
data("countries")
countries <- countries[countries != "Kenya"]
countries.filter <- gsub("[[:punct:]]", "[[:punct:]]", toupper(countries))
countries.filter <- gsub(" ", "[[:space:]]", countries.filter)
countries.filter <- paste("(",countries.filter,")", sep = "", collapse = "|")
countries.filter <- paste("(UK)|", countries.filter, sep = "")
cat(countries.filter)

(UK)|(AFGHANISTAN)|(ALBANIA)|(ALGERIA)|(ANDORRA)|(ANGOLA)|(ANTIGUA[[:space:]]AND[[:space:]]BARBUDA)|(ARGENTINA)|(ARMENIA)|(AUSTRALIA)|(AUSTRIA)|(AZERBAIJAN)|(BAHAMAS)|(BAHRAIN)|(BANGLADESH)|(BARBADOS)|(BELARUS)|(BELGIUM)|(BELIZE)|(BENIN)|(BHUTAN)|(BOLIVIA)|(BOSNIA[[:space:]]AND[[:space:]]HERZEGOVINA)|(BOTSWANA)|(BRAZIL)|(BRUNEI)|(BULGARIA)|(BURKINA[[:space:]]FASO)|(BURUNDI)|(CABO[[:space:]]VERDE)|(CAMBODIA)|(CAMEROON)|(CANADA)|(CENTRAL[[:space:]]AFRICAN[[:space:]]REPUBLIC)|(CHAD)|(CHILE)|(CHINA)|(COLOMBIA)|(COMOROS)|(CONGO)|(CONGO)|(COSTA[[:space:]]RICA)|(COTE[[:space:]]D[[:punct:]]IVOIRE)|(CROATIA)|(CUBA)|(CYPRUS)|(CZECH[[:space:]]REPUBLIC)|(DENMARK)|(DJIBOUTI)|(DOMINICA)|(DOMINICAN[[:space:]]REPUBLIC)|(ECUADOR)|(EGYPT)|(EL[[:space:]]SALVADOR)|(EQUATORIAL[[:space:]]GUINEA)|(ERITREA)|(ESTONIA)|(ETHIOPIA)|(FIJI)|(FINLAND)|(FRANCE)|(GABON)|(GAMBIA)|(GEORGIA)|(GERMANY)|(GHANA)|(GREECE)|(GRENADA)|(GUATEMALA)|(GUINEA)|(GUINEA[[:punct:]]BISSAU)|(GUYANA)|(HAITI)|(HONDURAS)|(HUNGARY)|(ICELAND)

In [7]:
# Show filter string
cat(paste(substr(countries.filter, 1, 85), "...", sep = ""))

(UK)|(AFGHANISTAN)|(ALBANIA)|(ALGERIA)|(ANDORRA)|(ANGOLA)|(ANTIGUA[[:space:]]AND[[:sp...

# Extract relevant summary data for the papers and save to file
The function was extracted from [businesspubmed](https://github.com/dami82/businessPubMed) to retrieve the various information such as "title", "year", "journal", "lastname", "firstname", "address", "email", "pmid", "month","day" from our query

In [8]:
#function for extracting pubmed information
extract_pubMed_data <- function (pubMed_query, 
                                 batch_size = 1000, 
                                 getKeywords = FALSE, 
                                 affi_regex_exclude = NULL) 
{
  ptm <- proc.time()
  my.idlist <- get_pubmed_ids(pubMed_query)
  record.num <- my.idlist$Count
  my.seq <- seq(1, as.numeric(my.idlist$Count), by = batch_size)
  pubmed.data <- lapply(my.seq, (function(ret.start) {
    batch.xml <- NULL
    message(paste("round #", which(my.seq == ret.start), 
                  " of ", length(my.seq), " ", sep = ""), appendLF = FALSE)
    while (is.null(batch.xml)) {
      batch.xml <- tryCatch({
        tmp.idlist <- get_pubmed_ids(pubMed_query)
        fetch_pubmed_data(tmp.idlist, retstart = ret.start, 
                          retmax = batch_size)
      }, error = function(e) {
        NULL
      })
    }
    record.list <- easyPubMed::articles_to_list(batch.xml)
    xtracted.data <- lapply(1:length(record.list), (function(i) {
      if (length(record.list) > 60) {
        custom.seq <- as.integer(seq(1, length(record.list), 
                                     length.out = 50))
        if (i %in% custom.seq) {
          message(".", appendLF = FALSE)
        }
      } else {
        message(".", appendLF = FALSE)
      }
      if (as.numeric(installed.packages()["easyPubMed","Version"]) > 2.5){
        tmp.record <- tryCatch(article_to_df(pubmedArticle = record.list[[i]], 
                                             getKeywords = getKeywords, 
                                             autofill = TRUE, max_chars = 10), 
                               error = function(e) {
                                 NULL
                               })
        required.cols <- c("title", "year", "journal", 
                           "lastname", "firstname", "address", "email", "pmid", "month","day")
        
        
      } else {
        tmp.record <- tryCatch(article_to_df(pubmedArticle = record.list[[i]], 
                                             autofill = TRUE, max_chars = 10), 
                               error = function(e) {
                                 NULL
                               })
        required.cols <- c("title", "year", "journal", 
                           "lastname", "firstname", "address", "email","pmid", "month", "day")
        
      }
      
      if (!is.null(tmp.record)) {
        out.record <- data.frame(matrix(NA, nrow = nrow(tmp.record), 
                                        ncol = length(required.cols)))
        colnames(out.record) <- required.cols
        match.cols <- colnames(tmp.record)[colnames(tmp.record) %in% 
                                             required.cols]
        out.record[, match.cols] <- tmp.record[, match.cols]
      } else {
        out.record <- NULL
      }
      out.record
    }))
    xtracted.data <- do.call(rbind, xtracted.data)
    message(" Filtering... ", appendLF = FALSE)
    xtracted.data <- xtracted.data[!is.na(xtracted.data$address), 
                                   ]
    if (!is.null(affi_regex_exclude)) {
      xtracted.data <- xtracted.data[regexpr(affi_regex_exclude, 
                                             xtracted.data$address,
                                             ignore.case = TRUE) < 0, ]
    }
    message("done!", appendLF = TRUE)
    xtracted.data
  }))
  stop.watch <- proc.time() - ptm
  pubmed.data <- do.call(rbind, pubmed.data)
  out.data <- list()
  out.data$params <- list()
  out.data$params$query_string <- pubMed_query
  out.data$params$pubMed_id_list <- my.idlist
  out.data$params$batch_size <- batch_size
  out.data$params$affi_regex_exclude <- affi_regex_exclude
  out.data$params$timing <- stop.watch
  out.data$data <- pubmed.data
  return(out.data)
}

we used the function *extract_pubMed_data* to gain the summary of the data using the regrex expression designed to filter out countries that are not kenyans 

In [9]:
#intial time 
t_init <- Sys.time()

#extract data from PubMed
my.data3 <- extract_pubMed_data(pubMed_query = my.query, 
                                batch_size = 1000,
                                affi_regex_exclude = countries.filter)
# Check time t_final
t_final <- Sys.time()

round #1 of 17 .................................................. Filtering... done!
round #2 of 17 .................................................. Filtering... done!
round #3 of 17 .................................................. Filtering... done!
round #4 of 17 .................................................. Filtering... done!
round #5 of 17 .................................................. Filtering... done!
round #6 of 17 .................................................. Filtering... done!
round #7 of 17 .................................................. Filtering... done!
round #8 of 17 .................................................. Filtering... done!
round #9 of 17 .................................................. Filtering... done!
round #10 of 17 .................................................. Filtering... done!
round #11 of 17 .................................................. Filtering... done!
round #12 of 17 ...............................................

The time taken for the code to run 

In [11]:
# processing time 
t_final -t_init

Time difference of 1.478131 hours

In [12]:
#few data set acquired 
head(my.data3$data, 10)

,title,year,journal,lastname,firstname,address,email,pmid,month,day
1,Current Trends of Nanobiosensors for Point-of-Care Diagnostics.,2019,Journal of analytical methods in chemistry,Noah,Naumih M,"School of Pharmacy and Health Sciences, United States International University-Africa, P.O. Box 14634-00800, Nairobi, Kenya",NA,31886019,1,1
2,Current Trends of Nanobiosensors for Point-of-Care Diagnostics.,2019,Journal of analytical methods in chemistry,Ndangili,Peter M,"Department of Chemical Science and Technology (DCST), Technical University of Kenya, P.O. Box 52428-00200, Nairobi, Kenya",NA,31886019,1,1
3,"Efficacy of Ivermectin, Liquid Paraffin, and Carbaryl against Mange of Farmed Rabbits in Central Kenya.",2019,Journal of tropical medicine,Ogolla,Kennedy O,"Department of Veterinary Pathology, Microbiology and Parasitology, University of Nairobi, P.O. Box 29053-00625, Kangemi, Nairobi, Kenya",NA,31885634,1,1
4,"Efficacy of Ivermectin, Liquid Paraffin, and Carbaryl against Mange of Farmed Rabbits in Central Kenya.",2019,Journal of tropical medicine,Chebet,Joyce,"Department of Veterinary Pathology, Microbiology and Parasitology, University of Nairobi, P.O. Box 29053-00625, Kangemi, Nairobi, Kenya",NA,31885634,1,1
5,"Efficacy of Ivermectin, Liquid Paraffin, and Carbaryl against Mange of Farmed Rabbits in Central Kenya.",2019,Journal of tropical medicine,Waruiru,Robert M,"Department of Veterinary Pathology, Microbiology and Parasitology, University of Nairobi, P.O. Box 29053-00625, Kangemi, Nairobi, Kenya",NA,31885634,1,1
6,"Efficacy of Ivermectin, Liquid Paraffin, and Carbaryl against Mange of Farmed Rabbits in Central Kenya.",2019,Journal of tropical medicine,Gathumbi,Peter K,"Department of Veterinary Pathology, Microbiology and Parasitology, University of Nairobi, P.O. Box 29053-00625, Kangemi, Nairobi, Kenya",NA,31885634,1,1
7,"Efficacy of Ivermectin, Liquid Paraffin, and Carbaryl against Mange of Farmed Rabbits in Central Kenya.",2019,Journal of tropical medicine,Okumu,Paul O,"Department of Veterinary Pathology, Microbiology and Parasitology, University of Nairobi, P.O. Box 29053-00625, Kangemi, Nairobi, Kenya",NA,31885634,1,1
8,"Efficacy of Ivermectin, Liquid Paraffin, and Carbaryl against Mange of Farmed Rabbits in Central Kenya.",2019,Journal of tropical medicine,Aboge,Gabriel O,"Department of Public Health, Pharmacology and Toxicology, University of Nairobi, P.O. Box 29053-00625, Kangemi, Nairobi, Kenya",NA,31885634,1,1
12,UHC2030's Contributions to Global Health Governance that Advance the Right to Health Care: A Preliminary Assessment.,2019,Health and human rights,Maleche,Allan,"Executive Director of the Kenya Legal & Ethical Issues Network on HIV & AIDS (KELIN), and a member of the UNAIDS Human Rights Reference Group on HIV and Human Rights. is executive director of the Kenya Legal & Ethical Issues Network on HIV & AIDS (KELIN), Nairobi, Kenya",NA,31885453,12,1
13,"Implementation research for public sector mental health care scale-up (SMART-DAPPER): a sequential multiple, assignment randomized trial (SMART) of non-specialist-delivered psychotherapy and/or medication for major depressive disorder and posttraumatic stress disorder (DAPPER) integrated with outpatient care clinics at a county hospital in Kenya.",2019,BMC psychiatry,Levy,Rachel,"Medical School, University of California, San Francisco, CA, USA",NA,31883526,12,28


In [13]:
# exporting data from the R console to the desktop
write.table(my.data3$data, "../data/Kenyapapers.csv", row.names=F, sep ="\t")

In [17]:
#Bash script for extracting pmid
x <- "bash ../script/extract_pmid.sh"

cat(x)

bash ../script/extract_pmid.sh

In [18]:
#Ran the bash script in R
system(x)

In [19]:
#importing the pmid id in R
Kenya_pmid.csv <- read.csv("../data/Kenyapmid.csv")

## The number of papers that have authors affliated to kenyan institutions

The data represents the number of Kenya papers that can be retrieved from the ncbi database. Previsously we identified about 13,473 PMID indicating a sharp increase of about 1000 pmid articles 

In [20]:
# number of id available
count(Kenya_pmid.csv)

n
14488


In [21]:
#DESIGNING FUNCTION 
#fetch Pmid from entrez
fetch_PMID_data <- function(pmids, format = "xml", encoding = "UTF-8", verbose = TRUE) 
{
  
  # Hardcoded
  pmids.per.batch = 100
  
  # custom f(x)
  query_and_fetch <- function(x, format = format, encoding = encoding) {
    q0 <- paste(paste0(x, "[PMID]"), collapse = " OR ")
    
    q1 <- easyPubMed::get_pubmed_ids(pubmed_query_string = q0)
    r1 <- easyPubMed::fetch_pubmed_data(pubmed_id_list = q1, format = format, encoding = encoding)
    r2 <- easyPubMed::articles_to_list(pubmed_data = r1, encoding = encoding, simplify = FALSE)
    
    nm2 <- lapply(r2, function(xx) { 
      y <- xx[1]; y <- gsub("</PMID>.+$", "</PMID>", y); custom_grep(y, "PMID", format = "list")[[1]]
    })
    
    
    nm2 <- as.character(do.call(c, nm2))
    
    # Order
    nuORD <- as.numeric(sapply(x, function(z) {
      which(nm2 == z)
    }))      
    r3 <- r2[nuORD]
    return(r3)
  }
  
  # standardize input
  if(is.character(pmids)) {
    pmids <- pmids[!is.na(pmids)]
  } else if (is.list(pmids)) {
    pmids <- do.call(c, pmids)
    pmids <- as.character(pmids)
    pmids <- pmids[!is.na(pmids)]
  }
  
  # split ID in small batches
  out <- list()
  ti <- Sys.time() - 2
  
  if (length(pmids) < pmids.per.batch) {
    
    out <- list(query_and_fetch(pmids, format = format, encoding = encoding))
    
  } else {
    all_i <- seq(1, (length(pmids) - 1), by = pmids.per.batch)
    all_i <- c(all_i, (length(pmids) + 1))
    
    for(j in 1:(length(all_i) - 1)) {
      
      if (verbose)
        message(".", appendLF = FALSE)
      
      i0 <- all_i[j]
      i1 <- (all_i[(j+1)] - 1)
      tdf <- as.numeric(difftime(time1 = Sys.time(), time2 =  ti, units = "sec"))
      
      # check time
      if (tdf < 1) {
        Sys.sleep(time = (1 - tdf))
      } 
      
      # generate query string
      tmp <- pmids[i0:i1]
      tmp2 <- query_and_fetch(x = tmp, format = format, encoding = encoding)
      names(tmp2) <- NULL
      
      ti <- Sys.time()
      out[[length(out) + 1]] <- tmp2
    }
  }
  
  # loop over and recompose
  OUT <- list()
  for(l1 in 1:length(out)) {
    TMP <- out[[l1]]
    for(l2 in 1:length(TMP)) {
      OUT[[length(OUT) + 1]] <- TMP[[l2]]
    }
  }
  
  if(verbose) {
    message("", appendLF = TRUE)
    message("Done!", appendLF = TRUE)
  }
  
  return(OUT)
}   
#extract ids(doi,pmcid,pmid) from pubmed 
extract_article_ids <- function(pubmed_data_list) {
  
  x <- pubmed_data_list
  
  # Tmp f(x)
  my_grep <- function(X, idtype = "pubmed") {
    
    myPAT <- paste0("^.*", idtype)
    out <- tryCatch({
      if (!grepl(myPAT, X)) {
        Y <- NA
      } else {
        Y <- sub(myPAT, "", X)
        Y <- sub("</ArticleId>.*$", "", Y)
        Y <- sub("^.*>", "", Y)
      }
      Y
    }, error = function(e) NA)
    
    return(out)  
  }
  
  # dddd
  out <- lapply(x, function(xx) {
    y <- sub("</ArticleIdList>.+", "</ArticleIdList>", xx)
    y <- sub("^.+(<ArticleIdList.+$)", "\\1", y)
    
    data.frame(
      PMID = my_grep(X = y, idtype = "pubmed"),
      DOI = my_grep(X = y, idtype = "doi"), 
      PMCID = my_grep(X = y, idtype = "pmc"), 
      stringsAsFactors = FALSE)
  })

# Return
  out <- do.call(rbind, out)
  return(out)
}   

In [52]:
#identifying the pmids 
head(Kenya_pmid.csv)

Pmid
10022051
10024569
10030753
10048769
10048831
10065173


## Status of open access among kenyan article 
The scripts extracts pmcid information on all the 14488 papers and hence creating a table that contains the status of open access among kenyan article.

In [22]:
#Extract data from pmid in xml format 
kenya_pmid.xml <- fetch_PMID_data (my.data3$data$pmid)

......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
Done!


In [23]:
# Retrival of the pmcid 
Kenyapmcid2.csv <- extract_article_ids(kenya_pmid.xml)

# # exporting data from the R console to the desktop
write.table(Kenyapmcid2.csv, "../data/Kenyapmcid2.csv", row.names=F, sep ="\t")

In [24]:
#Bash script for designing the Kenyan article table
y <- "bash ../script/data_creation.sh"

cat(y)

bash ../script/data_creation.sh

In [25]:
#Ran the bash script in R
system(y)

In [26]:
#importing the data in R
PMID_PMC_Journal_Year_Kenya2.csv<- read.csv("../data/PMID_PMC_Journal_Year_Kenya2.csv")

In [27]:
head(PMID_PMC_Journal_Year_Kenya2.csv)

Pmid,Journal,Year,Month,Pmcid
10022051,Preventive veterinary medicine,1999,1,NA
10024569,Infection and immunity,1999,3,PMC96455
10030753,Veterinary parasitology,1999,2,NA
10048769,Journal of interferon & cytokine research : the official journal of the International Society for Interferon and Cytokine Research,1999,1,NA
10048831,International journal for parasitology,1999,1,NA
10065173,East African medical journal,1998,11,NA


In [28]:
#Bash script for data manipulation
z <- "bash ../script/script.sh"

cat(z)

bash ../script/script.sh

In [29]:
#Ran the bash script in R
system(z)

In [31]:
#importing the data in R
PMID_PMC_Journal_Year_Kenya3.csv<- read.csv("../data/PMID_PMC_Journal_Year_Kenya3.csv")

In [32]:
head(PMID_PMC_Journal_Year_Kenya3.csv)

Pmid,Journal,Year,Month,Pmcid,Status
10022051,Preventive veterinary medicine,1999,1,NA,closed
10024569,Infection and immunity,1999,3,PMC96455,open
10030753,Veterinary parasitology,1999,2,NA,closed
10048769,Journal of interferon & cytokine research : the official journal of the International Society for Interferon and Cytokine Research,1999,1,NA,closed
10048831,International journal for parasitology,1999,1,NA,closed
10065173,East African medical journal,1998,11,NA,closed
